In [1]:
%load_ext autoreload
%autoreload 2

import hashlib
import json
import os
import sys
import uuid
from copy import deepcopy
from datetime import datetime

import PIL.Image
import torch

# PYTHONPATH=PPTAgent/src:$PYTHONPATH
os.sys.path.append('./src')

# --- imports from your project ---
from FlagEmbedding import BGEM3FlagModel
from marker.models import create_model_dict
import induct
from llms import LLM, setup_models
import pptgen
from model_utils import get_image_model, parse_pdf
from multimodal import ImageLabler
from presentation import Presentation
from utils import Config, is_image_path, pjoin, ppt_to_images
from doc_handling import refine_document
from topic_gen import topic_generate

# load api key
with open('api_key.json', 'r') as f:
    api_key = json.load(f)
    openai_api_key = api_key['openai_api_key']

LLM_MODEL = LLM(model="gpt-4o-2024-11-20",
                api_key=openai_api_key)


RUNS_DIR = "runs"
STAGES = [
    "PPT Parsing",
    "PDF/Topic Parsing",
    "Slide Induction",
    "PPT Generation",
    "Success!",
]
CAPTION_PROMPT_PATH = "prompts/caption.txt"
# For demonstration, let's load exactly 1 model (instead of multiple).
NUM_MODELS = 1
DEVICE_COUNT = torch.cuda.device_count() or torch.mps.device_count()


# --- **Task Configuration** ---
output_dir = None
# pdf = "./examples/DeepSeek-R1-short.pdf"
pdf = None
# ppt = "./examples/default_template_unedit.pptx"
ppt = "./examples/default_template-5p.pptx"
topic = "Introduce the history of OpenAI"  # won't be used if pdf is provided
slides = 5

/Users/yyyang/miniconda3/envs/ppt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# If user doesn't specify output_dir, create a unique one.
if not output_dir:
    dt_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_id = str(uuid.uuid4())[:8]
    project_id = f"{dt_str}_{unique_id}"
    output_dir = os.path.join(RUNS_DIR, project_id)
else:
    project_id = output_dir.split('/')[-1]

# Basic checks
if pdf is None and topic is None:
    print("[ERROR] Must provide either --pdf or --topic.")
    sys.exit(1)
if pdf and not os.path.exists(pdf):
    print(f"[ERROR] PDF file not found: {pdf}")
    sys.exit(1)

print("[INFO] Setting up models...")
language_model = LLM_MODEL
vision_model = LLM_MODEL
setup_models(language_model, vision_model)

ppt_template_path=ppt
pdf_path=pdf
topic=topic
slides_count=slides
output_dir=output_dir

print("[DONE] Presentation generation complete.")

[INFO] Setting up models...


Primary models connected successfully: gpt-4o-2024-11-20 and gpt-4o-2024-11-20

[DONE] Presentation generation complete.


In [3]:
"""
Given a PPT template and either a PDF file or a topic string,
generate a final presentation (pptx).
"""

# Create the run folder
os.makedirs(output_dir, exist_ok=True)
print(f"[INFO] Using run directory: {output_dir}")

# -- 1. Load models (text, image, marker) --
# Here we do a simpler single-model approach
text_model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True, device=0)
image_model = get_image_model(device=0)
marker_model = create_model_dict(device=0, dtype=torch.float16)


[INFO] Using run directory: runs/20250202_215225_6eab0d6a


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 303202.70it/s]
/Users/yyyang/miniconda3/envs/ppt/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any is

Loaded layout model datalab-to/surya_layout on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded table recognition model vikp/surya_tablerec on device mps with dtype torch.float16
Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16


In [4]:
# -- 2. Prepare the config objects --
generation_config = Config(output_dir)
# We'll store the "pptx" in a unique subfolder for safety.
pptx_md5 = hashlib.md5(open(ppt_template_path, "rb").read()).hexdigest()
pptx_config = Config(pjoin(RUNS_DIR, project_id, "pptx", pptx_md5))
os.makedirs(pptx_config.RUN_DIR, exist_ok=True)

# If you want to copy the PPT template into the "pptx" folder for caching:
if not os.path.exists(pjoin(pptx_config.RUN_DIR, "source.pptx")):
    os.system(f"cp '{ppt_template_path}' '{pjoin(pptx_config.RUN_DIR, 'source.pptx')}'")

print(f"[INFO] PPT Template MD5: {pptx_md5}")


[INFO] PPT Template MD5: f94b6c959a5907804512df07ddfbbbd6


In [5]:
# -- 3. Parse the PPT Template into a Presentation object --
print("[STAGE] PPT Parsing")
presentation = Presentation.from_file(
    pjoin(pptx_config.RUN_DIR, "source.pptx"), pptx_config
)
ppt_image_folder = pjoin(pptx_config.RUN_DIR, "slide_images")

if not os.path.exists(ppt_image_folder) or len(os.listdir(ppt_image_folder)) == 0:
    ppt_to_images(pjoin(pptx_config.RUN_DIR, "source.pptx"), ppt_image_folder)
    # Because your original code expects the slides to be renamed to slide_{i:04d}.jpg,
    # handle index alignment & remove error slides if any:
    for err_idx, _ in presentation.error_history:
        err_path = pjoin(ppt_image_folder, f"slide_{err_idx:04d}.jpg")
        if os.path.exists(err_path):
            os.remove(err_path)
    # rename real_idx => slide_idx
    for i, slide in enumerate(presentation.slides, start=1):
        slide.slide_idx = i
        old_name = pjoin(ppt_image_folder, f"slide_{slide.real_idx:04d}.jpg")
        new_name = pjoin(ppt_image_folder, f"slide_{slide.slide_idx:04d}.jpg")
        if os.path.exists(old_name):
            os.rename(old_name, new_name)

# You may optionally caption each slide image:
labler = ImageLabler(vision_model=vision_model, presentation=presentation, config=pptx_config)
labler.caption_images()

[STAGE] PPT Parsing


{}

In [6]:
# -- 4. Parse PDF or use a text topic to get doc_json --
print("[STAGE] PDF/Topic Parsing")
if pdf_path:
    # We’ll store PDF results under runs/pdf/<md5>/...
    pdf_md5 = hashlib.md5(open(pdf_path, "rb").read()).hexdigest()
    parsedpdf_dir = pjoin(RUNS_DIR, project_id, "pdf", pdf_md5)
    os.makedirs(parsedpdf_dir, exist_ok=True)
    # If the refined document is not cached, parse the PDF text & refine
    refined_doc_json_path = pjoin(parsedpdf_dir, "refined_doc.json")
    if not os.path.exists(refined_doc_json_path):
        print("[INFO] Parsing PDF ...")
        text_content = parse_pdf(pdf_path, parsedpdf_dir, marker_model)
        # Optional: also caption images found in PDF
        caption_json_path = pjoin(parsedpdf_dir, "caption.json")
        if not os.path.exists(caption_json_path):
            with open(CAPTION_PROMPT_PATH, "r", encoding="utf-8") as f:
                caption_prompt = f.read()
            images_info = {}
            for k in os.listdir(parsedpdf_dir):
                if is_image_path(k):
                    img_path = pjoin(parsedpdf_dir, k)
                    try:
                        text_cap = vision_model(caption_prompt, [img_path])
                        with PIL.Image.open(img_path) as img:
                            size = img.size
                        images_info[img_path] = [text_cap, size]
                    except Exception as e:
                        print(f"[ERROR] Could not caption {k}: {str(e)}")
            with open(caption_json_path, "w", encoding="utf-8") as f:
                json.dump(images_info, f, ensure_ascii=False, indent=4)

        # Now refine the markdown doc => JSON
        doc_json = refine_document(language_model, text_content)
        json.dump(doc_json, open(refined_doc_json_path, "w"), indent=4)
    else:
        print("[INFO] Using cached refined_doc.json")
        doc_json = json.load(open(refined_doc_json_path, "r"))
    # Also load the PDF's image captions (if you want them for generation):
    caption_json_path = pjoin(parsedpdf_dir, "caption.json")
    images = json.load(open(caption_json_path)) if os.path.exists(caption_json_path) else {}
    
else:
    # If no PDF, we assume we have a text topic
    print(f"[INFO] Generating from topic: '{topic}'")
    pdf_md5 = topic  # just reuse the "pdf" variable for your code’s logic
    doc_json = topic_generate(language_model, topic)
    
    # save doc_json to file
    topic_doc_json_path = pjoin(RUNS_DIR, project_id, "text_topic", pdf_md5)
    os.makedirs(topic_doc_json_path, exist_ok=True)
    json.dump(doc_json, open(pjoin(topic_doc_json_path, "topic_doc.json"), "w"), indent=4)
    images = {}


[STAGE] PDF/Topic Parsing
[INFO] Generating from topic: 'Introduce the history of OpenAI'


sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are a document content divider and extractor specialist, expert in dividing and extracting content 
from various types of topics and reorganizing it into a two-level json format for later PPT generation.'}]}, 
'[{"role": "user", "content": [{"type": "text", "text": "\\nBased on given markdown topic, generate a JSON output 
for later PPT generation, make sure the output is concise and focused.\\n\\nPlease generate a detailed presentation
planning document about given topic. Detail to about 250 words. \\n\\nFollow the JSON format of the example 
output:\\n{\\n    \\"metadata\\": {\\n        \\"title\\": \\"title of document\\",\\n        \\"author\\": \\"name
of authors\\",\\n        \\"publish date\\": \\"date of publication\\",\\n        \\"organization\\": \\"name of 
organization\\"\\n    },\\n    \\"sections\\": [\\n        {\\n            \\"title\\": \\"title of section1\\",\\n
\\"subsections\\": [\\n                {\\n                    \\"title\\": \\"title of subsection1.1\\",\\n       
\\"content\\": \\"content of subsection1.1\\"\\n                },\\n                {\\n                    
\\"title\\": \\"title of subsection1.2\\",\\n                    \\"content\\": \\"content of subsection1.2\\"\\n  
}\\n            ]\\n        },\\n        {\\n            \\"title\\": \\"title of section2\\",\\n            
\\"subsections\\": [\\n                {\\n                    \\"title\\": \\"title of subsection2.1\\",\\n       
\\"content\\": \\"content of subsection2.1\\"\\n                }\\n            ]\\n        }\\n    
]\\n}\\n\\nGiven Topic: \\nIntroduce the history of OpenAI\\n\\nOutput:"}]}]']

response from gpt-4o-2024-11-20: ```json
{
    "metadata": {
        "title": "The History of OpenAI",
        "author": "OpenAI Documentation Team",
        "publish date": "October 2023",
        "organization": "OpenAI"
    },
    "sections": [
        {
            "title": "Formation and Vision",
            "subsections": [
                {
                    "title": "Founding and Motivation",
                    "content": "OpenAI was founded in December 2015 by Elon Musk, Sam Altman, Greg Brockman, Ilya 
Sutskever, Wojciech Zaremba, and John Schulman, among others. The organization was established to ensure artificial
general intelligence (AGI) benefits all humanity and to research AI with a focus on safety and ethical 
considerations."
                },
                {
                    "title": "Non-Profit Roots",
                    "content": "Initially set up as a non-profit, OpenAI aimed to prioritize broad societal benefit
over profit motives. Early projects focused on transparency, collaboration, and open access to AI advancements."
                }
            ]
        },
        {
            "title": "Key Milestones and Achievements",
            "subsections": [
                {
                    "title": "Technical Breakthroughs",
                    "content": "OpenAI made significant contributions to AI with the development of groundbreaking 
models like GPT series (Generative Pre-trained Transformers) and DALL·E. These models demonstrated state-of-the-art
capabilities in natural language processing and image generation."
                },
                {
                    "title": "Transition to OpenAI LP",
                    "content": "In 2019, OpenAI transitioned to a capped-profit model as OpenAI LP to attract 
funding while adhering to its mission. Microsoft became a key partner, investing $1 billion and providing 
computational resources."
                },
                {
                    "title": "AI Democratization",
                    "content": "OpenAI has prioritized making its technologies accessible through platforms such as
the OpenAI API, enabling developers and organizations to integrate AI into various applications while ensuring 
ethical use."
                }
            ]
        },
        {
            "title": "The Future of OpenAI",
            "subsections": [
                {
                    "title": "Continued Ethical Leadership",
                    "content": "OpenAI remains committed to advancing AGI responsibly, ensuring both societal 
benefit and safety protocols in AI development. Its vision includes collaborating with stakeholders globally to 
navigate the challenges and opportunities AGI presents."
                }
            ]
        }
    ]
}
```

In [7]:
# -- 5. Slide Induction (looking at the PPT structure) --
print("[STAGE] Slide Induction")
template_img_dir = pjoin(pptx_config.RUN_DIR, "template_images")
if not os.path.exists(template_img_dir) or len(os.listdir(template_img_dir)) == 0:
    # Save a stripped version of the PPT
    deepcopy(presentation).save(
        pjoin(pptx_config.RUN_DIR, "template.pptx"), layout_only=False
    )
    ppt_to_images(
        pjoin(pptx_config.RUN_DIR, "template.pptx"), template_img_dir
    )

template_presentation = Presentation.from_file(
    pjoin(pptx_config.RUN_DIR, "template.pptx"), pptx_config
)

slide_inducter = induct.SlideInducter(
    vision_model,
    language_model,
    presentation,
    ppt_image_folder,
    template_img_dir,
    pptx_config,
    image_model,
    "inference_script",  # or "backend", up to you
)
slide_induction = slide_inducter.content_induct()
print(f"Slide Induction:\n{slide_induction}")

[STAGE] Slide Induction


sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are an expert presentation analyst specializing in categorizing PowerPoint slides, particularly 
skilled at identifying structural slides (such as Opening, Transitions, and Ending slides) that guide the flow of 
the presentation.'}]}, '[{"role": "user", "content": [{"type": "text", "text": "\\nInstructions:\\n\\t1.\\tAnalyze 
provided slides and identify structural slides (such as Opening and Ending) based on their 
content.\\n\\t2.\\tCategory names for structural slides should be simple, reflect their function, and not contain 
any specific details(such as the name of any entity).\\n\\t3.\\tOpening and Ending slides are typically located at 
the beginning or end of the presentation and may consist of only one slide.\\n\\t4.\\tOther structural categories 
should include multiple slides that share similar text elements (e.g., titles or \\"Table of Contents\\"). 
Moreover, they should not convey any specific information and serve as a guide for the presentation.\\n\\nExample 
output:\\n```json\\n{\\n    \\"opening\\": [1],\\n    \\"table of contents\\": [2, 5],\\n    \\"ending\\": 
[10]\\n}\\n```\\n\\nInput: Slide 1 of 5\\nTitle:Tourism & Culture:\\nTourism & Culture:\\nAPPRECIATING THE TANGIBLE
& THE INTANGIBLE OF BHUBANESWAR THROUGH CULTURAL TOURISM\\nBy Ayona Bhaduri\\n----\\nSlide 2 of 5\\nTitle:What is 
culture?\\nWhat is culture?\\nIn Sri Aurobindo’s words,\\nThe culture of a people may be described as the 
expression of a consciousness of life which formulates itself in three aspects.\\nThere is a side of thought, of 
ideal, of upward will and soul’s aspiration, which is reflected in the people’s philosophy and religion.\\nThere is
a side of creative self-expression and appreciative aesthesis, intelligence and imagination, which finds expression
in its art, poetry, literature, music, dance\\nAnd, there is a side of practical and outward formulation, which is 
provided for in its society and politics\\n----\\nSlide 3 of 5\\nTitle:What is Tourism?\\nWhat is Tourism?\\nThe 
World Tourism Organisation defines tourism in terms which go\\n“ beyond the common perception of tourism as being 
limited to holiday activity only”, as people “ travelling to and staying in places outside their usual environment 
for not more than one consecutive year for leisure, business and other purposes”\\nIts importance was recognised in
the Manila Declaration on World Tourism of 1980 as an activity essential to the life of nations because of its 
direct effects on the social, cultural, educational, and economic sectors of national societies and on their 
international relations\\nToday, tourism is a major source of income for many countries, and affects the economy of
both the source and the host countries, in some cases being of vital importance\\n----\\nSlide 4 of 5\\nTitle:What 
do we mean by Cultural Tourism?\\nWhat do we mean by Cultural Tourism?\\nCultural Tourism is the subset of tourism 
concerned with a particular country or region’s culture, specifically the lifestyle of the people in those 
geographical areas, the history of those people, their art and craft, their language and literature, their 
architecture, their religion, and all other elements that have helped shape their way of life.\\n\\nEvery race has 
something essential to contribute to the world’s civilisation in the course of its own self-expression and 
self-realisation. \\nAnd the essential contribution of India is simply her Indianness. \\nAll that India has to 
offer to the world proceeds from her philosophy. \\nThat this philosophy has held India together and seen her 
through many upheavals – political, social, economic, intellectual, is brought to the fore when Max Muller, the 
famous scholar and Orientalist, remarks, ‘There is, in fact, an unbroken continuity between the most modern and the
most ancient phases of Hindu thought, extending over mo

response from gpt-4o-2024-11-20: ```json
{
    "opening": [1]
}
```

sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are a helpful assistant'}]}, '[{"role": "user", "content": [{"type": "text", "text": "Analyze the 
content layout and media types in the provided slide images.\\nYour objective is to provide a concise, descriptive 
title that captures purely the layout pattern.\\nRequirements:\\nFocus on HOW content is structured and presented, 
not WHAT the content is\\nDescribe the visual arrangement and interaction between different content types (text, 
images, diagrams, etc.)\\n\\nAvoid:\\nSpecific topics or subjects\\nDetailed content descriptions\\n\\n\\nYou 
cannot use the following layout names:\\n[\'opening:text\']\\n\\nExample Outputs:\\nCentral Image with explanatory 
text\\nGrowth Overview Illustrated with the Chart\\nPicture and illustrative key points\\nPlain Bullet 
Points\\n\\nOutput: Provide a one-line layout pattern title."}, {"type": "image_url", "image_url": {"url": 
"data:image/jpeg;base64,<BASE64_IMAGE_DATA_REDACTED>"}}]}]']

response from gpt-4o-2024-11-20: Text-Centered Slide with Bullet Points

sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are a helpful assistant'}]}, '[{"role": "user", "content": [{"type": "text", "text": "Please analyze 
the slide elements and create a structured template schema in JSON format. The schema should:\\n\\n1. Identify key 
content elements (both text and images) that make up the slide\\n2. For each element, specify:\\n   - \\"name\\": 
The role of the element, such as \\"main title\\", \\"content bullets\\", \\"main image\\", \\"presenters\\", 
\\"background information\\", \\"acknowledgments\\", \\"data\\", etc.\\n   - \\"description\\": A clear description
of the element\'s purpose, do not mention any detail\\n   - \\"type\\": Literal[\\"text\\", \\"image\\"]\\n   - 
\\"data\\":\\n      * For text elements: The actual text content as string or array in paragraph level(<p> or 
<li>)\\n      * For image elements: Use the `alt` attribute of the <img> tag as the data of the image\\n\\nExample 
format:\\n{\\n  \\"element_name\\": {\\n    \\"description\\": \\"purpose of this element\\", # do not mention any 
detail, just purpose\\n    \\"type\\": \\"text\\" or \\"image\\",\\n    \\"data\\": [\\"text1\\", \\"text2\\"] or 
[\\"logo:...\\", \\"logo:...\\"]\\n  }\\n}\\nInput:\\n<!DOCTYPE html>\\n<html>\\n<title>Tourism & 
Culture:</title>\\n<body style=\\"width:960pt; height:540pt;\\">\\n<div>\\n\\t<p>Tourism & 
Culture:</p>\\n</div>\\n\\n<div>\\n\\t<p style=\'font-size: 24pt\'>APPRECIATING THE TANGIBLE & THE INTANGIBLE OF 
BHUBANESWAR THROUGH CULTURAL TOURISM</p>\\n\\t<p style=\'font-size: 20pt\'>By Ayona Bhaduri 
</p>\\n</div>\\n</body>\\n</html>\\n\\n\\nOutput: Please provide a schema that could be used as a template for 
creating similar slides"}]}]']

response from gpt-4o-2024-11-20: Here is the structured JSON schema template based on the provided slide content. 
It identifies the key content elements, their purpose, content type, and data:

```json
{
  "slide_title": {
    "description": "The main title describing the subject of the slide.",
    "type": "text",
    "data": ["Tourism & Culture:"]
  },
  "main_heading": {
    "description": "The central or main heading that introduces the core theme or message of the slide.",
    "type": "text",
    "data": ["APPRECIATING THE TANGIBLE & THE INTANGIBLE OF BHUBANESWAR THROUGH CULTURAL TOURISM"]
  },
  "presenter": {
    "description": "The name of the person or entity presenting the content of the slide.",
    "type": "text",
    "data": ["By Ayona Bhaduri"]
  }
}
```

This JSON schema template serves as a guide to creating similar slides by identifying essential elements and their 
corresponding purposes. The `"data"` field should be updated with appropriate content as needed for new slides.

sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are a helpful assistant'}]}, '[{"role": "user", "content": [{"type": "text", "text": "Please analyze 
the slide elements and create a structured template schema in JSON format. The schema should:\\n\\n1. Identify key 
content elements (both text and images) that make up the slide\\n2. For each element, specify:\\n   - \\"name\\": 
The role of the element, such as \\"main title\\", \\"content bullets\\", \\"main image\\", \\"presenters\\", 
\\"background information\\", \\"acknowledgments\\", \\"data\\", etc.\\n   - \\"description\\": A clear description
of the element\'s purpose, do not mention any detail\\n   - \\"type\\": Literal[\\"text\\", \\"image\\"]\\n   - 
\\"data\\":\\n      * For text elements: The actual text content as string or array in paragraph level(<p> or 
<li>)\\n      * For image elements: Use the `alt` attribute of the <img> tag as the data of the image\\n\\nExample 
format:\\n{\\n  \\"element_name\\": {\\n    \\"description\\": \\"purpose of this element\\", # do not mention any 
detail, just purpose\\n    \\"type\\": \\"text\\" or \\"image\\",\\n    \\"data\\": [\\"text1\\", \\"text2\\"] or 
[\\"logo:...\\", \\"logo:...\\"]\\n  }\\n}\\nInput:\\n<!DOCTYPE html>\\n<html>\\n<title>What is 
Tourism?</title>\\n<body style=\\"width:960pt; height:540pt;\\">\\n<div>\\n\\t<p>What is 
Tourism?</p>\\n</div>\\n\\n<div style=\'font-size: 20pt\'>\\n\\t<p>The World Tourism Organisation defines tourism 
in terms which go</p>\\n\\t<p>“ beyond the common perception of tourism as being limited to holiday activity only”,
as people “ travelling to and staying in places outside their usual environment for not more than one consecutive 
year for leisure, business and other purposes”</p>\\n\\t<p>Its importance was recognised in the Manila Declaration 
on World Tourism of 1980 as an activity essential to the life of nations because of its direct effects on the 
social, cultural, educational, and economic sectors of national societies and on their international 
relations</p>\\n\\t<p>Today, tourism is a major source of income for many countries, and affects the economy of 
both the source and the host countries, in some cases being of vital 
importance</p>\\n</div>\\n</body>\\n</html>\\n\\n\\nOutput: Please provide a schema that could be used as a 
template for creating similar slides"}]}]']

response from gpt-4o-2024-11-20: Here is a structured JSON schema for creating similar slides based on the provided
HTML input.

```json
{
  "main_title": {
    "description": "The primary heading that introduces the main topic of the slide",
    "type": "text",
    "data": ["What is Tourism?"]
  },
  "content_text": {
    "description": "The body text providing detailed explanations, definitions, or information related to the 
topic",
    "type": "text",
    "data": [
      "The World Tourism Organisation defines tourism in terms which go",
      "“ beyond the common perception of tourism as being limited to holiday activity only”, as people “ travelling
to and staying in places outside their usual environment for not more than one consecutive year for leisure, 
business and other purposes”",
      "Its importance was recognised in the Manila Declaration on World Tourism of 1980 as an activity essential to
the life of nations because of its direct effects on the social, cultural, educational, and economic sectors of 
national societies and on their international relations",
      "Today, tourism is a major source of income for many countries, and affects the economy of both the source 
and the host countries, in some cases being of vital importance"
    ]
  }
}
```

### Explanation
1. **main_title**: Represents the slide's main title, extracted from the first `<p>` element.
2. **content_text**: Represents the body text, composed of multiple `<p>` paragraphs that elaborate on the topic.

This schema can serve as a baseline for structuring similar slides, ensuring consistent organization and clear 
specification of elements. If future slides include images or additional structured content, corresponding new 
elements can be added to the schema.

Slide Induction:
defaultdict(<function SlideInducter.__init__.<locals>.<lambda> at 0x3ea538360>, {'opening:text': defaultdict(<class 'list'>, {'slides': [1], 'template_id': 1, 'content_schema': {'slide_title': {'description': 'The main title describing the subject of the slide.', 'type': 'text', 'data': ['Tourism & Culture:']}, 'main_heading': {'description': 'The central or main heading that introduces the core theme or message of the slide.', 'type': 'text', 'data': ['APPRECIATING THE TANGIBLE & THE INTANGIBLE OF BHUBANESWAR THROUGH CULTURAL TOURISM']}, 'presenter': {'description': 'The name of the person or entity presenting the content of the slide.', 'type': 'text', 'data': ['By Ayona Bhaduri']}}}), 'Text-Centered Slide with Bullet Points:text': defaultdict(<class 'list'>, {'template_id': 3, 'slides': [3, 5, 4, 2], 'content_schema': {'main_title': {'description': 'The primary heading that introduces the main topic of the slide', 'type': 'text', 'data': ['What is Tourism?']}, 'cont

In [8]:
# -- 6. PPT Generation --
print("[STAGE] PPT Generation")
# instantiate the “crew” that handles text generation for slides
crew = pptgen.PPTCrew(vision_model, language_model, text_model,
                      error_exit=True, retry_times=3)

crew.set_reference(template_presentation, slide_induction, generation_config)

# Actually generate the new PPT
# (the code will produce final.pptx in output_dir)
crew.generate_pres(
    generation_config,
    images,         # PDF or custom images (captions, if any)
    slides_count,   # number of pages to generate
    doc_json        # the JSON structure
)

print("[STAGE] Success!")
print(f"[INFO] Output PPT stored at {pjoin(output_dir, 'final.pptx')}")

[STAGE] PPT Generation


sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are a professional presentation designer tasked with creating structured Presentation outlines. The 
final deliverable should be formatted as a JSON object. Please ensure that no layouts other than those provided are
utilized in the outline.\n'}]}, '[{"role": "user", "content": [{"type": "text", "text": "Instructions:\\nReview the
provided document overview, image captions, and available layouts.\\nThen, create a structured presentation outline
by following these guidelines:\\n  1. Ensure the number of slides matches the specified requirement.\\n  2. Use 
structural layouts only for structural slides to make the presentation more engaging (eg. use opening and ending 
slides once each).\\n  3. Analyze the content and media types specified in the provided layouts to optimize slide 
design.\\n  4. Utilize high-quality, topic-related images to make the presentation more engaging (use layouts with 
media type=image for related ).\\n\\nFor each slide, provide:\\n  - Slide Purpose: A high-level summary of the 
slide\'s objective, used as the key for the dictionary item.\\n  - Layout: Choose an appropriate layout from the 
provided options, tailored to the slide\'s purpose and media type.\\n  - Slide Description: A concise description 
of the slide\'s content.\\n  - Subsection Keys: Relevant subsection titles that relate to the slide\'s 
content.\\n\\nPlease provide your output in JSON format.\\n\\nThe Number of Slides in Your Output JSON MUST BE 
EQUAL to the Number of Slides: 5\\nFor the layouts, usually, first slide is the opening, last slide is the ending, 
and the other slides are the content slides (choose from the content layouts).\\n\\nExample Output for a 3-slide 
presentation:\\n{\\n   \\"Opening of the XX\\": {\\n      \\"layout\\": \\"opening(media_type)\\",\\n      
\\"subsections\\": [],\\n      \\"description\\": \\"...\\"\\n   },\\n   \\"Introduction to the XX\\": {\\n      
\\"layout\\": \\"layout1(media_type)\\", # select from given layouts(structural or content)\\n      
\\"subsections\\": [\\"Title of Subsection 1.1\\", \\"Title of Subsection 1.2\\"],\\n      \\"description\\": 
\\"...\\"\\n  },\\n  ...,\\n  \\"Ending of the XX\\": {\\n      \\"layout\\": \\"ending(media_type)\\",\\n      
\\"subsections\\": [],\\n      \\"description\\": \\"...\\"\\n  }\\n}\\n\\nInput:\\n{\'metadata\': {\'title\': 
\'The History of OpenAI\', \'author\': \'OpenAI Documentation Team\', \'publish date\': \'October 2023\', 
\'organization\': \'OpenAI\'}, \'sections\': [{\'title\': \'Formation and Vision\', \'subsections\': [{\'title\': 
\'Founding and Motivation\'}, {\'title\': \'Non-Profit Roots\'}]}, {\'title\': \'Key Milestones and Achievements\',
\'subsections\': [{\'title\': \'Technical Breakthroughs\'}, {\'title\': \'Transition to OpenAI LP\'}, {\'title\': 
\'AI Democratization\'}]}, {\'title\': \'The Future of OpenAI\', \'subsections\': [{\'title\': \'Continued Ethical 
Leadership\'}]}]}\\n\\nAvailable Images:\\n\\n\\n# you can only use the following layouts\\nContent 
Layouts:\\nText-Centered Slide with Bullet Points:text\\nfunctional_keys\\n\\nStructural 
Layouts:\\nopening:text\\n\\nOutput:"}]}]']

response from gpt-4o-2024-11-20: ```json
{
   "Opening of The History of OpenAI": {
      "layout": "opening:text",
      "subsections": [],
      "description": "An engaging introduction to the history and evolution of OpenAI, highlighting its 
foundational vision, key milestones, and future outlook."
   },
   "Formation and Vision": {
      "layout": "text-centered_slide",
      "subsections": ["Founding and Motivation", "Non-Profit Roots"],
      "description": "Detailing OpenAI's founding principles, motivations, and its beginnings as a non-profit 
organization."
   },
   "Key Milestones and Achievements": {
      "layout": "text-centered_slide",
      "subsections": ["Technical Breakthroughs", "Transition to OpenAI LP", "AI Democratization"],
      "description": "Exploring OpenAI's notable achievements, including technical innovations, organizational 
changes, and efforts to democratize access to artificial intelligence."
   },
   "The Future of OpenAI": {
      "layout": "text-centered_slide",
      "subsections": ["Continued Ethical Leadership"],
      "description": "A forward-looking analysis of OpenAI's commitment to ethical leadership in AI research and 
implementation."
   },
   "Conclusion of The History of OpenAI": {
      "layout": "ending:text",
      "subsections": [],
      "description": "Summarizing key takeaways and emphasizing OpenAI's ongoing mission to drive AI development 
responsibly and transparently."
   }
}
```

generating 5 slides, with 5 outlines

sending messages to: gpt-4o-2024-11-20, message (truncated): [{'role': 'system', 'content': [{'type': 'text', 
'text': 'You are an expert Editor agent. Transform reference text and images into slide content, following schema 
rules and using only provided materials. Ensure the content is engaging and within the character limit.'}]}, 
'[{"role": "user", "content": [{"type": "text", "text": "Task: Generate engaging slide content based on the 
provided schema and reference materials.\\n\\nRequirements:\\n1. Content Generation Rules:\\n  - Follow 
default_quantity for elements, adjust when necessary\\n  - Ensure text content meets character limits\\n  - 
Generated text should use concise and impactful presentation style\\n\\n2. Core Elements:\\n  - Must extract 
essential content from reference text (e.g., slide_title, main_content) and maintain semantic consistency\\n  - 
Must include images that support the main content (e.g., diagrams for explanations, visuals directly discussed in 
text)\\n\\n3. Supporting Elements (e.g., presenters, logo images):\\n  - Generate only when relevant content exists
in reference materials\\n\\nGenerate content for each element and output in the following format:\\n{\\n  
\\"element1\\": {\\n    \\"data\\": [\\"text1\\", \\"text2\\"] for text elements\\n          or 
[\\"/path/to/image\\", \\"...\\"] for image elements\\n  },\\n}\\n\\nInput:\\nSchema:\\n{\'slide_title\': 
{\'description\': \'The main title describing the subject of the slide.\', \'type\': \'text\', 
\'suggestedCharacters\': \'<18\', \'default_quantity\': 1}, \'main_heading\': {\'description\': \'The central or 
main heading that introduces the core theme or message of the slide.\', \'type\': \'text\', 
\'suggestedCharacters\': \'<82\', \'default_quantity\': 1}, \'presenter\': {\'description\': \'The name of the 
person or entity presenting the content of the slide.\', \'type\': \'text\', \'suggestedCharacters\': \'<16\', 
\'default_quantity\': 1}}\\n\\nMetadata of Presentation:\\ntitle: The History of OpenAI\\nauthor: OpenAI 
Documentation Team\\npublish date: October 2023\\norganization: OpenAI\\nPresentation Time: 2025-02-02\\n\\nSlide 
1: Opening of The History of OpenAI\\nSlide 2: Formation and Vision\\nSlide 3: Key Milestones and 
Achievements\\nSlide 4: The Future of OpenAI\\nSlide 5: Conclusion of The History of OpenAI\\n\\nReference 
Text:\\nSlide-1 Slide Purpose: Opening of The History of OpenAI\\nSlide Description: An engaging introduction to 
the history and evolution of OpenAI, highlighting its foundational vision, key milestones, and future 
outlook.\\n\\n\\nAvailable Images:\\nNo Images\\n\\nOutput: Ensure the generated content strictly adheres to the 
schema specifications, follows the slide style, reader-friendly."}]}]']

...

slide_layout_name Title and Content

slide_title Bhubaneswar – The Temple City of India

slide_width 960

slide_height 540

[STAGE] Success!
[INFO] Output PPT stored at runs/20250202_215225_6eab0d6a/final.pptx


### PPT gen test

In [9]:
for slide in presentation.slides:
    print(slide.slide_idx, slide.slide_title)
    for key, value in slide.__dict__.items():
        print(key, value)
    print("\n\n")

# presentation cut off test

presentation = Presentation.from_file("./examples/default_template_unedit.pptx", pptx_config)
presentation.slides = presentation.slides[:2]
presentation.save("cutoff.pptx")


1 Tourism & Culture:
shapes [TextBox: shape 0 of slide 1, TextBox: shape 1 of slide 1]
slide_idx 1
real_idx 1
background_xml <p:bgPr xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">
  <a:noFill/>
  <a:effectLst/>
</p:bgPr>

slide_notes None
slide_layout_name Title Slide
slide_title Tourism & Culture:
slide_width 960
slide_height 540



2 What is culture?
shapes [TextBox: shape 0 of slide 2, TextBox: shape 1 of slide 2]
slide_idx 2
real_idx 2
background_xml <p:bgPr xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">
  <a:noFill/>
  <a:effectLst/>
</p:bgPr>

slide_notes None
slide_layout_name Title and Content
slide_title What is culture?
slide_width 960
slide_height 540